In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
os.chdir('/Users/chrissoria/Documents/Research/BICS_Political_Polarization/')

In [3]:
col_types = {
    'FIPS_Code': str,
    'District': str,
    'CD':str
}

CD_df = pd.read_csv('data/House_Reps_District.csv', dtype=col_types)

some ZIP Codes, such as "unique" ZIPs that are often tied to an internal mail-routing mechanism (such as UC Berkeley which has its own internal ZIP code and mail-routing system) will have very low ratios). 

In [4]:
ZIP_to_CD_df = pd.read_excel('Data/Partisanship and Health Behavior/Data/ZIP_CD_062020.xlsx')

ZIP_to_CD_df = ZIP_to_CD_df.rename(columns={
    'RES_RATIO': 'CD_RES_RATIO',
    'BUS_RATIO': 'CD_BUS_RATIO',
    'OTH_RATIO': 'CD_OTH_RATIO',
    'TOT_RATIO': 'CD_TOT_RATIO'
})

ZIP_to_CD_df['ZIP'] = ZIP_to_CD_df['ZIP'].astype(str).str.zfill(5)

For now, I will assign a ZIP code to whatever whatever County it falls mostly under residentially

In [5]:
col_types = {
    'COUNTY': str,
    'ZIP': str
}

ZIP_to_County_df = pd.read_excel('Data/Partisanship and Health Behavior/Data/ZIP_COUNTY_062020_HUD.xlsx', dtype=col_types)
ZIP_to_County_df['COUNTY'] = ZIP_to_County_df['COUNTY'].astype(str).str.zfill(5)
ZIP_to_County_df['ZIP'] = ZIP_to_County_df['ZIP'].astype(str).str.zfill(5)

# Sort the DataFrame by RES_RATIO in descending order
ZIP_to_County_df = ZIP_to_County_df.sort_values(by='RES_RATIO', ascending=False)
ZIP_to_County_df = ZIP_to_County_df.drop_duplicates(subset='ZIP', keep='first')

ZIP_to_County_df.reset_index(drop=True, inplace=True)

ZIP_to_County_df = ZIP_to_County_df.rename(columns={
    'RES_RATIO': 'COUNTY_RES_RATIO',
    'BUS_RATIO': 'COUNTY_BUS_RATIO',
    'OTH_RATIO': 'COUNTY_OTH_RATIO',
    'TOT_RATIO': 'COUNTY_TOT_RATIO'
})

In [6]:
ZIP_FEATURES = pd.merge(ZIP_to_CD_df, ZIP_to_County_df, on = "ZIP", how = "left")
ZIP_FEATURES

,ZIP,CD,CD_RES_RATIO,CD_BUS_RATIO,CD_OTH_RATIO,CD_TOT_RATIO,COUNTY,COUNTY_RES_RATIO,COUNTY_BUS_RATIO,COUNTY_OTH_RATIO,COUNTY_TOT_RATIO
0,00501,3601,0.0,1.0,0.0,1.0,36103,0.000000,1.000000,0.000000,1.000000
1,00601,7200,1.0,1.0,1.0,1.0,72001,0.839242,0.800983,0.871166,0.837603
2,00602,7200,1.0,1.0,1.0,1.0,72003,1.000000,0.998801,1.000000,0.999919
3,00603,7200,1.0,1.0,1.0,1.0,72005,0.997152,0.998160,1.000000,0.997290
4,00604,7200,1.0,1.0,1.0,1.0,72005,1.000000,1.000000,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...
45909,99925,0200,0.0,0.0,1.0,1.0,02198,0.000000,0.000000,1.000000,1.000000
45910,99926,0200,0.0,0.0,1.0,1.0,02198,0.000000,0.000000,1.000000,1.000000
45911,99927,0200,0.0,0.0,1.0,1.0,02198,0.000000,0.000000,1.000000,1.000000
45912,99928,0200,0.0,0.0,1.0,1.0,02130,0.000000,0.000000,1.000000,1.000000


Maybe it doesn't make sense to have ZIP to CD and ZIP to County in the same dataset. I might make these into two later on, but for now this contains the county under which the ZIP code mostly falls for residential addresses. 

In [7]:
ZIP_FEATURES['CD'] = ZIP_FEATURES['CD'].astype(str)
CD_df['CD'] = CD_df['CD'].astype(str)


ZIP_FEATURES = pd.merge(ZIP_FEATURES, CD_df, on = "CD", how= "left")
ZIP_FEATURES = ZIP_FEATURES.drop(columns=['FIPS_Code','Years_Served','District','CD_RES_RATIO',
                                                      'CD_BUS_RATIO','CD_OTH_RATIO','COUNTY_RES_RATIO',
                                                      'COUNTY_BUS_RATIO','COUNTY_OTH_RATIO'])

ZIP_FEATURES = ZIP_FEATURES.rename(columns={
    'COUNTY': 'COUNTY_FIPS',
    'Name': 'CONGRESSPERSON',
    'State': 'STATE',
    'Party': 'CONGRESSPERSON_PARTY'
})

cols_to_move = ZIP_FEATURES.columns[3:5]
df_remaining = ZIP_FEATURES.drop(columns=cols_to_move)
ZIP_FEATURES = pd.concat([df_remaining, ZIP_FEATURES[cols_to_move]], axis=1)


ZIP_FEATURES

,ZIP,CD,CD_TOT_RATIO,CONGRESSPERSON,STATE,CONGRESSPERSON_PARTY,COUNTY_FIPS,COUNTY_TOT_RATIO
0,00501,3601,1.0,"Zeldin, Lee M.",New York,Republican,36103,1.000000
1,00601,7200,1.0,"González-Colón, Jenniffer",Puerto Rico,Republican,72001,0.837603
2,00602,7200,1.0,"González-Colón, Jenniffer",Puerto Rico,Republican,72003,0.999919
3,00603,7200,1.0,"González-Colón, Jenniffer",Puerto Rico,Republican,72005,0.997290
4,00604,7200,1.0,"González-Colón, Jenniffer",Puerto Rico,Republican,72005,1.000000
...,...,...,...,...,...,...,...,...
47722,99925,0200,1.0,"Young, Don",Alaska,Republican,02198,1.000000
47723,99926,0200,1.0,"Young, Don",Alaska,Republican,02198,1.000000
47724,99927,0200,1.0,"Young, Don",Alaska,Republican,02198,1.000000
47725,99928,0200,1.0,"Young, Don",Alaska,Republican,02130,1.000000


In [8]:
ZIP_FEATURES.to_csv('data/ZIP_Features.csv')

Pulling from their API to see if I can extract any more years worth of data \
It looks like it's only cumulative.

In [9]:
import requests
import json

# API endpoint
url = "https://data.cdc.gov/resource/kn79-hsxy.json"

# Making a GET request
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    data = response.json()
    
    # Extracting all 'data_as_of' values
    data_as_of_values = [item['data_as_of'] for item in data if 'data_as_of' in item]

    # Getting unique values by converting the list to a set
    unique_data_as_of_values = set(data_as_of_values)

    # Printing unique 'data_as_of' values
    print("Unique 'data_as_of' values:")
    for value in unique_data_as_of_values:
        print(value)
else:
    print(f"Failed to retrieve data. Status code: {response.status_code}")

Unique 'data_as_of' values:
2023-06-28T00:00:00.000


Below, we need to add a leading "0" to the FIPS code (FIPS codes are all 5 digits) \
I'll borrw Urban Rural Code from here

In [10]:
col_types = {
    'FIPS County Code': str
}

CDC_deaths = pd.read_csv('Data/Partisanship and Health Behavior/Data/Provisional_COVID-19_Death_Counts_in_the_United_States_by_County.csv', dtype=col_types)
CDC_deaths['FIPS County Code'] = CDC_deaths['FIPS County Code'].astype(str).str.zfill(5)

CDC_deaths = CDC_deaths.drop(columns = ['Date as of','Deaths from All Causes','Footnote','State'])
CDC_deaths.head()

,Start Date,End Date,County name,FIPS County Code,Urban Rural Code,Deaths involving COVID-19
0,01/01/2020,06/24/2023,Aleutians East Borough,02013,Noncore,NaN
1,01/01/2020,06/24/2023,Anchorage Municipality,02020,Medium metro,775.0
2,01/01/2020,06/24/2023,Bethel Census Area,02050,Noncore,40.0
3,01/01/2020,06/24/2023,Denali Borough,02068,Noncore,NaN
4,01/01/2020,06/24/2023,Dillingham Census Area,02070,Noncore,NaN


In [11]:
col_types = {
    'countyFIPS': str
}

usafacts_deaths = pd.read_csv('Data/Partisanship and Health Behavior/Data/covid_deaths_usafacts.csv', dtype=col_types)
usafacts_deaths['countyFIPS'] = usafacts_deaths['countyFIPS'].astype(str).str.zfill(5)

For this preliminary analysis, I will only use wave 2,4,6. \
Each wave's data was collected on 2020-06-17 - 2020-06-23, 2020-11-29 - 2020-12-16, 2021-05-12 - 2021-05-25. \
I will take covid mortality from the midpoint of the wave range. \
Wave 2 = 6/20/2020 \
Wave 4 = 12/9/2020 \
Wave 6 = 5/19/2021

In [12]:
columns_to_keep = ['countyFIPS', 'County Name', '2020-06-20', '2020-12-09', '2021-05-19']

usafacts_deaths = usafacts_deaths[columns_to_keep]

usafacts_deaths = usafacts_deaths.rename(columns={
    'countyFIPS': 'COUNTY_FIPS',
    '2020-06-20': 'COUNTY_COV_DEATHS_2020_06_20',
    '2020-12-09': 'COUNTY_COV_DEATHS_2020_12_09',
    '2021-05-19': 'COUNTY_COV_DEATHS_2021_05_19'
})

In [13]:
ZIP_FEATURES = pd.merge(ZIP_FEATURES, usafacts_deaths, on = "COUNTY_FIPS", how = "left")
ZIP_FEATURES

,ZIP,CD,CD_TOT_RATIO,CONGRESSPERSON,STATE,CONGRESSPERSON_PARTY,COUNTY_FIPS,COUNTY_TOT_RATIO,County Name,COUNTY_COV_DEATHS_2020_06_20,COUNTY_COV_DEATHS_2020_12_09,COUNTY_COV_DEATHS_2021_05_19
0,00501,3601,1.0,"Zeldin, Lee M.",New York,Republican,36103,1.000000,Suffolk County,1963.0,2093.0,3383.0
1,00601,7200,1.0,"González-Colón, Jenniffer",Puerto Rico,Republican,72001,0.837603,NaN,NaN,NaN,NaN
2,00602,7200,1.0,"González-Colón, Jenniffer",Puerto Rico,Republican,72003,0.999919,NaN,NaN,NaN,NaN
3,00603,7200,1.0,"González-Colón, Jenniffer",Puerto Rico,Republican,72005,0.997290,NaN,NaN,NaN,NaN
4,00604,7200,1.0,"González-Colón, Jenniffer",Puerto Rico,Republican,72005,1.000000,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
47722,99925,0200,1.0,"Young, Don",Alaska,Republican,02198,1.000000,Prince of Wales - Hyder Census Area,0.0,1.0,1.0
47723,99926,0200,1.0,"Young, Don",Alaska,Republican,02198,1.000000,Prince of Wales - Hyder Census Area,0.0,1.0,1.0
47724,99927,0200,1.0,"Young, Don",Alaska,Republican,02198,1.000000,Prince of Wales - Hyder Census Area,0.0,1.0,1.0
47725,99928,0200,1.0,"Young, Don",Alaska,Republican,02130,1.000000,Ketchikan Gateway Borough,0.0,0.0,1.0


In [14]:
col_types1 = {
    'STATE': str,
    'COUNTY': str
}

col_types2 = {
    'County Code (FIPS)': str,
    'County Subdivision Code (FIPS)': str,
    'Place Code (FIPS)': str,
    'Consolidtated City Code (FIPS)': str,
    'State Code (FIPS)': str
}


census_pop = pd.read_csv('Data/Partisanship and Health Behavior/Data/co-est2021-alldata.csv', encoding='ISO-8859-1',
                        dtype = col_types1)
census_pop_FIPS = pd.read_excel('Data/Partisanship and Health Behavior/Data/all-geocodes-v2020.xlsx', 
                                dtype = col_types2)

census_pop_FIPS = census_pop_FIPS.rename(columns={
    'Area Name (including legal/statistical area description)': 'CTYNAME'
})

census_pop['COUNTY_FIPS'] = census_pop['STATE']+census_pop['COUNTY']

columns_to_keep = ['STATE','POPESTIMATE2020','DEATHS2020','POPESTIMATE2021','DEATHS2021','COUNTY_FIPS']

census_pop = census_pop[columns_to_keep]

census_pop = census_pop.rename(columns={
    'STATE': 'STATE_FIPS'
})

census_pop

,STATE_FIPS,POPESTIMATE2020,DEATHS2020,POPESTIMATE2021,DEATHS2021,COUNTY_FIPS
0,01,5024803,16148,5039877,64868,01000
1,01,58877,168,59095,681,01001
2,01,233140,661,239294,2867,01003
3,01,25180,109,24964,394,01005
4,01,22223,90,22477,282,01007
...,...,...,...,...,...,...
3189,56,42158,116,41614,400,56037
3190,56,23347,29,23575,96,56039
3191,56,20441,49,20635,166,56041
3192,56,7658,32,7705,115,56043


A crude mortality rate, not standardized or age adjusted, for this preliminary analysis

In [15]:
ZIP_FEATURES = pd.merge(ZIP_FEATURES, census_pop, on = "COUNTY_FIPS", how = "left")


ZIP_FEATURES['COV_County_CMR_2020_06_20'] = (ZIP_FEATURES['COUNTY_COV_DEATHS_2020_06_20'] / ZIP_FEATURES['POPESTIMATE2020']) * 1000
ZIP_FEATURES['COV_County_CMR_2020_12_09'] = (ZIP_FEATURES['COUNTY_COV_DEATHS_2020_12_09'] / ZIP_FEATURES['POPESTIMATE2020']) * 1000
ZIP_FEATURES['COV_County_CMR_2021_05_19'] = (ZIP_FEATURES['COUNTY_COV_DEATHS_2021_05_19'] / ZIP_FEATURES['POPESTIMATE2021']) * 1000
ZIP_FEATURES

,ZIP,CD,CD_TOT_RATIO,CONGRESSPERSON,STATE,CONGRESSPERSON_PARTY,COUNTY_FIPS,COUNTY_TOT_RATIO,County Name,COUNTY_COV_DEATHS_2020_06_20,COUNTY_COV_DEATHS_2020_12_09,COUNTY_COV_DEATHS_2021_05_19,STATE_FIPS,POPESTIMATE2020,DEATHS2020,POPESTIMATE2021,DEATHS2021,COV_County_CMR_2020_06_20,COV_County_CMR_2020_12_09,COV_County_CMR_2021_05_19
0,00501,3601,1.0,"Zeldin, Lee M.",New York,Republican,36103,1.000000,Suffolk County,1963.0,2093.0,3383.0,36,1524099.0,3806.0,1526344.0,15459.0,1.287974,1.373270,2.216407
1,00601,7200,1.0,"González-Colón, Jenniffer",Puerto Rico,Republican,72001,0.837603,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00602,7200,1.0,"González-Colón, Jenniffer",Puerto Rico,Republican,72003,0.999919,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00603,7200,1.0,"González-Colón, Jenniffer",Puerto Rico,Republican,72005,0.997290,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00604,7200,1.0,"González-Colón, Jenniffer",Puerto Rico,Republican,72005,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47722,99925,0200,1.0,"Young, Don",Alaska,Republican,02198,1.000000,Prince of Wales - Hyder Census Area,0.0,1.0,1.0,02,5725.0,26.0,5729.0,36.0,0.000000,0.174672,0.174551
47723,99926,0200,1.0,"Young, Don",Alaska,Republican,02198,1.000000,Prince of Wales - Hyder Census Area,0.0,1.0,1.0,02,5725.0,26.0,5729.0,36.0,0.000000,0.174672,0.174551
47724,99927,0200,1.0,"Young, Don",Alaska,Republican,02198,1.000000,Prince of Wales - Hyder Census Area,0.0,1.0,1.0,02,5725.0,26.0,5729.0,36.0,0.000000,0.174672,0.174551
47725,99928,0200,1.0,"Young, Don",Alaska,Republican,02130,1.000000,Ketchikan Gateway Borough,0.0,0.0,1.0,02,13902.0,26.0,13754.0,106.0,0.000000,0.000000,0.072706


In [16]:
ZIP_FEATURES.to_csv('data/ZIP_Features.csv', index = False)

Why is resp_educ and education different?